# ÁP DỤNG RANDOM FOREST XÂY DỰNG MÔ HÌNH DỰ ĐOÁN ĐIỂM THI

- Mục tiêu: PredictScore

In [253]:
# Import các thư viện cần thiết cho mô hình
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from scipy.stats import zscore

In [254]:
# Đọc dữ liệu từ dataset
data = pd.read_csv('dataset.csv', index_col= "Id")

data.head()

,Age,Grade,StudyTime,OnlineCourse,SchoolType,ClassType,SelfStudy,GoingOut,Health,LatestScore,...,AverageExerciseScore,TestsCompleted,AverageTestScore,AverageTestCompletionTime,ArenaParticipation,ArenaQuestionsAnsweredPercentage,BestOfArenaCount,AverageArenaRanking,MostTime,PredictScore
Id,,,,,,,,,,,,,,,,,,,,,
1,18,13,3,no,specialized,normal,5,4,5,10,...,1.79,34,1.08,95.25,20,42.23,1,5,day,6.34
2,18,11,1,no,public,gifted,4,2,5,3,...,8.97,9,8.69,86.48,17,37.11,0,5,night,7.30
3,16,13,4,yes,public,gifted,1,5,3,4,...,6.10,48,7.81,82.81,10,26.17,3,5,day,7.46
4,16,10,1,no,specialized,normal,2,1,5,4,...,7.67,20,6.27,97.93,4,49.69,0,5,night,7.12
5,16,13,1,no,private,gifted,2,5,4,6,...,8.72,46,4.24,99.95,3,21.15,0,3,day,6.21


In [255]:
# Kiểm tra dữ liệu thiếu
missing_values = data.isnull().sum()
print(missing_values)

Age                                 0
Grade                               0
StudyTime                           0
OnlineCourse                        0
SchoolType                          0
ClassType                           0
SelfStudy                           0
GoingOut                            0
Health                              0
LatestScore                         0
DailyTimeSpent                      0
DayActive                           0
ResourceAccessed                    0
ForumParticipationFrequency         0
HelpfulAnswers                      0
ExercisesCompleted                  0
AverageExerciseScore                0
TestsCompleted                      0
AverageTestScore                    0
AverageTestCompletionTime           0
ArenaParticipation                  0
ArenaQuestionsAnsweredPercentage    0
BestOfArenaCount                    0
AverageArenaRanking                 0
MostTime                            0
PredictScore                        0
dtype: int64

In [256]:
# Mã hóa các biến phân loại
categorical_columns = ['OnlineCourse', 'SchoolType', 'ClassType', 'MostTime']
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le
print("\nDữ liệu sau khi mã hóa các biến phân loại:")
print(data[categorical_columns])


Dữ liệu sau khi mã hóa các biến phân loại:
      OnlineCourse  SchoolType  ClassType  MostTime
Id                                                 
1                0           2          1         0
2                0           1          0         1
3                1           1          0         0
4                0           2          1         1
5                0           0          0         0
...            ...         ...        ...       ...
4037             1           1          0         1
4038             0           0          1         1
4039             0           2          1         1
4040             0           1          0         1
4041             1           0          1         1

[4041 rows x 4 columns]


In [257]:
# Chuẩn hóa các đặc trưng và xóa bỏ ngoại lệ

numerical_columns = ['Age', 'Grade', 'StudyTime', 'SelfStudy', 'GoingOut', 'Health', 'AverageTestScore', 'AverageExerciseScore', 'LatestScore', 'TestsCompleted']
scaler = StandardScaler()
data[numerical_columns] = scaler.fit_transform(data[numerical_columns])

data = data[(np.abs(zscore(data[numerical_columns])) < 3).all(axis=1)]
print(data.head())


         Age     Grade  StudyTime  OnlineCourse  SchoolType  ClassType  \
Id                                                                       
1  -0.006060  1.343754   0.032665             0           2          1   
2  -0.006060 -0.449545  -1.386700             0           1          0   
3  -1.819942  1.343754   0.742348             1           1          0   
4  -1.819942 -1.346195  -1.386700             0           2          1   
5  -1.819942  1.343754  -1.386700             0           0          0   

    SelfStudy  GoingOut    Health  LatestScore  ...  AverageExerciseScore  \
Id                                              ...                         
1    1.406266  0.719493  1.429955     1.571463  ...             -1.125290   
2    0.704349 -0.693616  1.429955    -0.640848  ...              1.368283   
3   -1.401402  1.426047  0.015056    -0.324804  ...              0.371548   
4   -0.699485 -1.400170  1.429955    -0.324804  ...              0.916800   
5   -0.699485  1.42

In [258]:
# Tách đặc trưng và biến mục tiêu

features = [col for col in data.columns if col != 'PredictScore']
X = data[features]
y = data['PredictScore']
print("\nCác đặc trưng được chọn:")
print(features)
print("\nBiến mục tiêu:")
print(y.head())



Các đặc trưng được chọn:
['Age', 'Grade', 'StudyTime', 'OnlineCourse', 'SchoolType', 'ClassType', 'SelfStudy', 'GoingOut', 'Health', 'LatestScore', 'DailyTimeSpent', 'DayActive', 'ResourceAccessed', 'ForumParticipationFrequency', 'HelpfulAnswers', 'ExercisesCompleted', 'AverageExerciseScore', 'TestsCompleted', 'AverageTestScore', 'AverageTestCompletionTime', 'ArenaParticipation', 'ArenaQuestionsAnsweredPercentage', 'BestOfArenaCount', 'AverageArenaRanking', 'MostTime']

Biến mục tiêu:
Id
1    6.34
2    7.30
3    7.46
4    7.12
5    6.21
Name: PredictScore, dtype: float64


In [259]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("\nKích thước tập huấn luyện và tập kiểm tra:")
print("X_train:", X_train.shape)
print("X_test:", X_test.shape)
print("y_train:", y_train.shape)
print("y_test:", y_test.shape)
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)



Kích thước tập huấn luyện và tập kiểm tra:
X_train: (3232, 25)
X_test: (809, 25)
y_train: (3232,)
y_test: (809,)


In [266]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Dự đoán trên tập kiểm tra
y_pred = model.predict(X_test)

# Đánh giá mô hình
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'R²: {r2}')

MSE: 0.09005562945611864
MAE: 0.2407400494437577
R²: 0.9186146673801926


In [261]:
features

['Age',
 'Grade',
 'StudyTime',
 'OnlineCourse',
 'SchoolType',
 'ClassType',
 'SelfStudy',
 'GoingOut',
 'Health',
 'LatestScore',
 'DailyTimeSpent',
 'DayActive',
 'ResourceAccessed',
 'ForumParticipationFrequency',
 'HelpfulAnswers',
 'ExercisesCompleted',
 'AverageExerciseScore',
 'TestsCompleted',
 'AverageTestScore',
 'AverageTestCompletionTime',
 'ArenaParticipation',
 'ArenaQuestionsAnsweredPercentage',
 'BestOfArenaCount',
 'AverageArenaRanking',
 'MostTime']

In [262]:
reality = pd.DataFrame([
    {
        'Age': 17,
        'Grade': 11,
        'StudyTime': 4,
        'OnlineCourse': 1,
        'SchoolType': 'specialized',
        'ClassType': 'gifted',
        'SelfStudy': 2,
        'GoingOut': 4,
        'Health': 5,
        'LatestScore': 10,
        'DailyTimeSpent': 4.1,
        'DayActive': 11,
        'ResourceAccessed': 22,
        'ForumParticipationFrequency': 1,
        'HelpfulAnswers': 12,
        'ExercisesCompleted': 27,
        'AverageExerciseScore': 4.9,
        'TestsCompleted': 34,
        'AverageTestScore': 9.2,
        'AverageTestCompletionTime': 76.2,
        'ArenaParticipation': 19,
        'ArenaQuestionsAnsweredPercentage': 33.56,
        'BestOfArenaCount': 0,
        'AverageArenaRanking': 2,
        'MostTime': 'day'
    }
])

reality.head()

,Age,Grade,StudyTime,OnlineCourse,SchoolType,ClassType,SelfStudy,GoingOut,Health,LatestScore,...,ExercisesCompleted,AverageExerciseScore,TestsCompleted,AverageTestScore,AverageTestCompletionTime,ArenaParticipation,ArenaQuestionsAnsweredPercentage,BestOfArenaCount,AverageArenaRanking,MostTime
0,17,11,4,1,specialized,gifted,2,4,5,10,...,27,4.9,34,9.2,76.2,19,33.56,0,2,day


In [263]:
reality

,Age,Grade,StudyTime,OnlineCourse,SchoolType,ClassType,SelfStudy,GoingOut,Health,LatestScore,...,ExercisesCompleted,AverageExerciseScore,TestsCompleted,AverageTestScore,AverageTestCompletionTime,ArenaParticipation,ArenaQuestionsAnsweredPercentage,BestOfArenaCount,AverageArenaRanking,MostTime
0,17,11,4,1,specialized,gifted,2,4,5,10,...,27,4.9,34,9.2,76.2,19,33.56,0,2,day


In [264]:
ct_col = ['OnlineCourse', 'SchoolType', 'ClassType', 'MostTime']
lb_encoders = {}
for col in ct_col:
    le = LabelEncoder()
    reality[col] = le.fit_transform(reality[col])
    lb_encoders[col] = le
print("\nDữ liệu sau khi mã hóa các biến phân loại:")
print(reality[ct_col])

# nm_col = ['Age', 'Grade', 'StudyTime', 'SelfStudy', 'GoingOut', 'Health', 'AverageTestScore', 'AverageExerciseScore', 'LatestScore', 'TestsCompleted']
# scalers = StandardScaler()
# data[numerical_columns] = scaler.fit_transform(re[numerical_columns])

# data = data[(np.abs(zscore(data[numerical_columns])) < 3).all(axis=1)]
# print(data.head())


Dữ liệu sau khi mã hóa các biến phân loại:
   OnlineCourse  SchoolType  ClassType  MostTime
0             0           0          0         0


In [265]:
import pandas as pd
import numpy as np

# Giả sử rằng các bộ mã hóa và chuẩn hóa đã được lưu trữ trong biến label_encoders và scaler
# Cần sử dụng lại các bộ mã hóa và chuẩn hóa đã được huấn luyện trên dữ liệu ban đầu

# Dữ liệu thực tế mới
reality = pd.DataFrame([
    {
        'Age': 17,
        'Grade': 11,
        'StudyTime': 8,
        'OnlineCourse': 'yes',
        'SchoolType': 'specialized',
        'ClassType': 'gifted',
        'SelfStudy': 5,
        'GoingOut': 1,
        'Health': 5,
        'LatestScore': 10,
        'DailyTimeSpent': 4.1,
        'DayActive': 11,
        'ResourceAccessed': 50,
        'ForumParticipationFrequency': 5,
        'HelpfulAnswers': 12,
        'ExercisesCompleted': 27,
        'AverageExerciseScore': 10,
        'TestsCompleted': 34,
        'AverageTestScore': 10,
        'AverageTestCompletionTime': 76.2,
        'ArenaParticipation': 50,
        'ArenaQuestionsAnsweredPercentage': 55.56,
        'BestOfArenaCount': 4,
        'AverageArenaRanking': 2,
        'MostTime': 'day'
    }
])

print("Dữ liệu thực tế ban đầu:")
print(reality.head())

# Mã hóa các biến phân loại
for col in categorical_columns:
    reality[col] = label_encoders[col].transform(reality[col])

print("\nDữ liệu sau khi mã hóa các biến phân loại:")
print(reality.head())

# Chuẩn hóa các đặc trưng số học
reality[numerical_columns] = scaler.transform(reality[numerical_columns])

print("\nDữ liệu sau khi chuẩn hóa các đặc trưng số học:")
print(reality.head())

# Chọn các đặc trưng đã sử dụng trong mô hình
reality_features = reality[features]

# Dự đoán với mô hình Random Forest đã huấn luyện
predicted_score = model.predict(reality_features)
print("\nDự đoán điểm thi cho dữ liệu thực tế mới:")
print(predicted_score)

Dữ liệu thực tế ban đầu:
   Age  Grade  StudyTime OnlineCourse   SchoolType ClassType  SelfStudy  \
0   17     11          8          yes  specialized    gifted          5   

   GoingOut  Health  LatestScore  ...  ExercisesCompleted  \
0         1       5           10  ...                  27   

   AverageExerciseScore  TestsCompleted  AverageTestScore  \
0                    10              34                10   

   AverageTestCompletionTime  ArenaParticipation  \
0                       76.2                  50   

   ArenaQuestionsAnsweredPercentage  BestOfArenaCount  AverageArenaRanking  \
0                             55.56                 4                    2   

   MostTime  
0       day  

[1 rows x 25 columns]

Dữ liệu sau khi mã hóa các biến phân loại:
   Age  Grade  StudyTime  OnlineCourse  SchoolType  ClassType  SelfStudy  \
0   17     11          8             1           2          0          5   

   GoingOut  Health  LatestScore  ...  ExercisesCompleted  \
0      